In [1]:
%%capture
# run this cell if this package is not installed
# !pip install nbformat

In [1]:
%%capture
%run setup.ipynb

In [2]:
# Define a function to remove p1 and p2
def remove_start_end(text):
    
    p1 = "Chat Conversation Start"
    p2 = "Chat Conversation End"
    
    # Remove p1 only the first time it appears
    text = re.sub(rf'^\s*{re.escape(p1)}\s*', '', text)

    # Remove p2 only the last time it appears
    text = re.sub(rf'\s*{re.escape(p2)}\s*$', '', text)

    return text

# Apply the function to the "Content" column
df['Content'] = df['Content'].apply(remove_start_end)

To do: fix this function as it doesn't remove the emails as it should 

In [69]:
def extract_email_addresses(text):
    pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b'
    emails = re.findall(pattern, text)
    return emails if emails else None

def remove_emails(text, emails):
    if emails is not None:
        for email in emails:
            text = re.sub(re.escape(email), '', text)
    return text

def normalize_document(doc):
    wpt = nltk.WordPunctTokenizer()
    stop_words = set(nltk.corpus.stopwords.words('english'))

    # lowercase and remove special characters\whitespace
    doc = re.sub(r'[^a-zA-Z\s]', '', str(doc), re.I | re.A)
    doc = doc.lower()
    doc = doc.strip()
    
    # extract email addresses using a more robust regex
    emails = extract_email_addresses(doc)

    # remove email addresses from the document
    doc = remove_emails(doc, emails)
            
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    
    return doc

# Function to apply cleaning to an entire DataFrame
def clean_dataframe(df):
    # Apply the cleaning function to each cell in the DataFrame
    cleaned_df = df.applymap(lambda x: normalize_document(x))
    return cleaned_df


In [70]:
emails = df['Content'].apply(extract_email_addresses)

remove_emails(df['Content'], emails)

TypeError: decoding to str: need a bytes-like object, NoneType found

In [63]:
clean_dataframe(df)

,File_Name,Content
0,txt,u good girls bed haha omg miss lot lovely frie...
1,txt,u hi u great thanks kind fine think problem as...
2,txt,u hello u hello hello best friend finally real...
3,txt,u want ask study lot mathematics business engi...
4,txt,u hi checked courses ask friends see one took ...
...,...,...
11331,txt,psyrobtwubvmsdccbuffaloedu robert weiss subjec...
11332,txt,sandviknewtonapplecom kent sandvik subject apr...
11333,txt,sandviknewtonapplecom kent sandvik subject dis...
11334,txt,cutterglosterviamindorg cutter subject biblica...
